![SETI](https://earthsky.org/upl/2020/02/Earth-transit-zone-Breakthrough-Listen.jpg)

# About This Notebook

This is simply the Inference Notebook for models I have trained separately.
* Training Notebook:- [SETI : EDA and Pytorch Starter](https://www.kaggle.com/manabendrarout/nfnet-pytorch-starter-lb-0-95)

**If you found this notebook useful and use parts of it in your work, please don't forget to show your appreciation by upvoting this kernel. That keeps me motivated and inspires me to write and share these public kernels.** 😊

In [ ]:
import sys
sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

In [ ]:
# Asthetics
import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

# General
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import random
import glob
pd.set_option('display.max_columns', None)

# Image Aug
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

# Deep Learning
import torch
import torchvision
import timm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

# Random Seed Initialize
RANDOM_SEED = 42

def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything()

# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print(f'Using device: {device}')

In [ ]:
csv_dir = '../input/seti-breakthrough-listen'
test_dir = '../input/seti-breakthrough-listen/test'

sample_sub_file_path = os.path.join(csv_dir, 'sample_submission.csv')
print(f'Train file: {sample_sub_file_path}')

test_df = pd.read_csv(sample_sub_file_path)

In [ ]:
def return_filpath(name, folder=test_dir):
    path = os.path.join(folder, name[0], f'{name}.npy')
    return path

In [ ]:
test_df['image_path'] = test_df['id'].apply(lambda x: return_filpath(x))
test_df.head()

# Model

In [ ]:
params = {
    'model': 'nfnet_l0',
    'inp_channels': 1,
    'device': device,
    'batch_size': 32,
    'num_workers' : 0,
    'out_features': 1,
    'path': '../input/seti-pytorch-models/nfnet_l0_6_epoch_0.98_roc_auc.pth',
    'TTA': 3
}

## Image Augmentations

In [ ]:
def get_test_transforms(TTA):
    '''
    Returns an augmented Image if the TTA count is > 1 otherwise
    returns the resized image only.
    '''
    if TTA > 1:
        return albumentations.Compose(
            [
                albumentations.Resize(256,256),
                albumentations.HorizontalFlip(p=0.5),
                albumentations.VerticalFlip(p=0.5),
                albumentations.Rotate(limit=180, p=0.7),
                albumentations.RandomBrightness(limit=0.6, p=0.5),
                albumentations.ShiftScaleRotate(
                    shift_limit=0.25, scale_limit=0.1, rotate_limit=0
                ),
                ToTensorV2(p=1.0),
            ]
        )
    
    else:
        return albumentations.Compose(
            [
                albumentations.Resize(256,256),
                ToTensorV2(p=1.0)
            ]
        )

## Dataset Class

In [ ]:
class SETIDataset(Dataset):
    def __init__(self, images_filepaths, targets, transform=None):
        self.images_filepaths = images_filepaths
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = np.load(image_filepath)
        image = image.astype(np.float32)
        image = np.vstack(image).transpose((1, 0))
            
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        else:
            image = image[np.newaxis,:,:]
            image = torch.from_numpy(image).float()
        
        label = torch.tensor(self.targets[idx]).float()
        return image, label

## CNN Model

In [ ]:
class AlienNet(nn.Module):
    def __init__(self, model_name=params['model'], out_features=params['out_features'],
                 inp_channels=params['inp_channels'], pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained,
                                       in_chans=inp_channels)
        if model_name.split('_')[0] == 'efficientnet':
            n_features = self.model.classifier.in_features
            self.model.conv_stem = nn.Conv2d(inp_channels, 40, kernel_size=(3, 3),
                                             stride=(2, 2), padding=(1, 1), bias=False)
            self.model.classifier = nn.Linear(n_features, out_features)
        
        elif model_name.split('_')[0] == 'nfnet':
            n_features = self.model.head.fc.in_features
            self.model.head.fc = nn.Linear(n_features, out_features)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = AlienNet()
model.load_state_dict(torch.load(params['path'], map_location=device))
model = model.to(params['device'])

## Prediction

In [ ]:
model.eval()
predicted_labels = None
for i in range(params['TTA']):
    test_dataset = SETIDataset(
        images_filepaths = test_df['image_path'].values,
        targets = test_df['target'].values,
        transform=get_test_transforms(params['TTA'])
    )
    test_loader = DataLoader(
        test_dataset, batch_size=params['batch_size'],
        shuffle=False, num_workers=params['num_workers']
    )
    
    temp_preds = None
    with torch.no_grad():
        for (images, target) in tqdm(test_loader):
            images = images.to(params['device'], non_blocking=True)
            output = model(images)
            predictions = torch.sigmoid(output).cpu().numpy()
            if temp_preds is None:
                temp_preds = predictions
            else:
                temp_preds = np.vstack((temp_preds, predictions))
    
    if predicted_labels is None:
        predicted_labels = temp_preds
    else:
        predicted_labels += temp_preds
        
predicted_labels /= params['TTA']

In [ ]:
sub_df = pd.DataFrame()
sub_df['id'] = test_df['id']
sub_df['target'] = predicted_labels

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('submission.csv', index=False)

This is a simple starter kernel on implementation of Transfer Learning using Pytorch for this problem. Pytorch has many SOTA Image models which you can try out using the guidelines in this notebook.

I hope you have learnt something from this notebook. I have created this notebook as a baseline model, which you can easily fork and paly-around with to get much better results. I might update parts of it down the line when I get more GPU hours and some interesting ideas.

**If you liked this notebook and use parts of it in you code, please show some support by upvoting this kernel. It keeps me inspired to come-up with such starter kernels and share it with the community.**

Thanks and happy kaggling!